# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [6]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [55]:
step1 = data.groupby(['CustomerID', 'ProductName'], as_index=False)['Quantity'].agg('sum')

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [23]:
matrix = data.pivot_table(values='Quantity', index=['CustomerID'],columns=['ProductName'], aggfunc=np.sum, fill_value=0)
matrix.T.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [74]:
distances = pd.DataFrame(1/(1 + squareform(pdist(matrix, 'euclidean'))), 
                         index=matrix.index, columns=matrix.index)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [33]:
data["33"].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 33, dtype: float64

In [36]:
distances = pd.DataFrame(1/(1 + squareform(pdist(matrix, 'euclidean'))), 
                         index=matrix.index, columns=matrix.index)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [63]:
similarities = distances.iloc[1:,].sort_values(by='CustomerID', ascending = False)[1:6]
similarities

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
98185,0.004167,0.004166,0.004179,0.004175,0.004177,0.004167,0.004168,0.004176,0.004172,0.004161,...,0.003311,0.003162,0.003255,0.003406,0.003172,0.003255,0.003345,0.003277,1.000000,0.003288
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
97900,0.004312,0.004311,0.004322,0.004310,0.004318,0.004308,0.004321,0.004308,0.004314,0.004314,...,0.003431,0.003357,0.003234,0.003509,1.000000,0.003579,0.003496,0.003322,0.003172,0.003288


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [56]:
step1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [66]:
amiguis = [98185,98159,98069,97928,97900]
top5 = step1.loc[step1.CustomerID.isin(amiguis)]
top5.head()

,CustomerID,ProductName,Quantity
63225,97900,Appetizer - Sausage Rolls,25
63226,97900,Apricots - Halves,25
63227,97900,"Bar Mix - Pina Colada, 355 Ml",25
63228,97900,Beef - Inside Round,25
63229,97900,Beef - Prime Rib Aaa,25


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [77]:
cpr = top5.groupby(['ProductName'], as_index=False)["Quantity"].agg('sum').sort_values(by="Quantity",ascending = False).head()

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [75]:
matrix2 = matrix.T.reset_index()
matrix2

CustomerID,ProductName,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,...,0,25,0,0,0,0,0,0,0,0
1,"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
2,Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,25,0
3,Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
4,Apricots - Dried,1,0,0,0,1,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
5,Apricots - Halves,0,0,1,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
6,Apricots Fresh,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,25
7,Arizona - Green Tea,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,25,0,0,0,0
8,Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Assorted Desserts,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,25,0,0,25


In [78]:
matrix2 = matrix2.merge(cpr, left_on='ProductName', right_on='ProductName')
matrix2

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Apricots - Halves,0,0,1,0,0,0,0,0,0,...,0,0,0,25,50,25,0,25,25,125
1,Flour - Teff,0,0,1,0,0,0,0,0,1,...,0,0,0,50,25,0,25,0,0,100
2,Juice - Orange,0,0,0,0,0,0,0,0,0,...,0,0,25,25,25,0,25,25,0,100
3,Ketchup - Tomato,0,0,0,0,0,1,0,1,1,...,0,0,0,25,25,25,25,0,0,100
4,Scallops - Live In Shell,0,0,0,1,0,0,0,1,0,...,0,0,0,25,25,25,0,50,0,125


In [79]:
matrix2 = matrix2.sort_values(by='Quantity', ascending=False)
matrix2

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Apricots - Halves,0,0,1,0,0,0,0,0,0,...,0,0,0,25,50,25,0,25,25,125
4,Scallops - Live In Shell,0,0,0,1,0,0,0,1,0,...,0,0,0,25,25,25,0,50,0,125
1,Flour - Teff,0,0,1,0,0,0,0,0,1,...,0,0,0,50,25,0,25,0,0,100
2,Juice - Orange,0,0,0,0,0,0,0,0,0,...,0,0,25,25,25,0,25,25,0,100
3,Ketchup - Tomato,0,0,0,0,0,1,0,1,1,...,0,0,0,25,25,25,25,0,0,100


In [80]:
matrix2[['ProductName',33,'Quantity']].loc[matrix2[33]==0].head()

,ProductName,33,Quantity
0,Apricots - Halves,0,125
4,Scallops - Live In Shell,0,125
1,Flour - Teff,0,100
2,Juice - Orange,0,100
3,Ketchup - Tomato,0,100


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.